# IAM MFA USERS

This notebook will show users who do/not have MFA enabled.

It will also show users that are in the `MFARequired` Group, but have their MFA enabled (and should be removed from `MFARequired`).

Finally it will remove those users from `MFARequired`

In [ ]:
import sys
!{sys.executable} -m pip install boto3
import boto3

In [ ]:
profile = {"name": "ss-dev", "region": "us-east-1"}
boto3.setup_default_session(profile_name=profile["name"], region_name=profile["region"])
client = boto3.client('iam')

In [ ]:
def remove_bots(x):
    name = x.get("UserName")
    bots= ["circleci", "MissionOkta", "looker", "autoreport"]
    bad_chars = ["-"]
    
    if name not in bots and "." not in name:
        if "-" not in name and "_" not in name:
            return True
    if "@" in name:
        return True
    return False

users =list(filter(remove_bots, client.list_users()["Users"]))
usernames = [x.get("UserName") for x in users]
print(usernames)

In [ ]:
# no_mfa = { name: False for name in usernames if len(client.list_mfa_devices(UserName=name)["MFADevices"]) == 0 }
mfa = { name: True if len(client.list_mfa_devices(UserName=name)["MFADevices"]) > 0 else False for name in usernames }
# print(no_mfa)
print(mfa)

In [ ]:
group_users_to_remove = []
for key,value in mfa.items():
    if value:
        try:
            for group in client.list_groups_for_user(UserName=key)["Groups"]:
                if group.get("GroupName") == "MFARequired":
                    group_users_to_remove.append(key)
                    
        except:
            print("{} {}".format(key, no_mfa[key]))

# print(no_mfa)
print(group_users_to_remove)

In [ ]:
for user in group_users_to_remove:
    user = client.remove_user_from_group(GroupName="MFARequired", UserName=user)